# Chapter 14: Risk and Reality Checks
AFML focus: tail risk, drawdowns, and probability-adjusted performance scrutiny.


In [1]:
import math
import matplotlib.pyplot as plt
import polars as pl
import openquant
import sys
from pathlib import Path
sys.path.insert(0, str(Path('notebooks/python/scripts').resolve()))
from afml_chapter_utils import (
    fetch_panel,
    simple_returns,
    probs_and_sides_from_momentum,
    timestamps_from_dates,
    lag_corr,
    fracdiff_ffd,
)

panel = fetch_panel(window=900)
dates = panel['date'].to_list()
uso = panel['USO'].to_list()
uso_ret = simple_returns(uso)
probs, sides = probs_and_sides_from_momentum(uso)
timestamps = timestamps_from_dates(dates)
asset_names = ['USO', 'BNO', 'XLE', 'GLD', 'UNG']
asset_prices = panel.select(asset_names).rows()
print('rows', panel.height, 'range', dates[0], dates[-1])


rows 900 range 2022-07-08 2026-02-06


In [2]:
out = openquant.pipeline.run_mid_frequency_pipeline_frames(
    timestamps=timestamps,
    close=uso,
    model_probabilities=probs,
    model_sides=sides,
    asset_prices=asset_prices,
    asset_names=asset_names,
)

bt = out['backtest']
var95 = openquant.risk.calculate_value_at_risk(bt['strategy_returns'], 0.05)
es95 = openquant.risk.calculate_expected_shortfall(bt['strategy_returns'], 0.05)
cdar95 = openquant.risk.calculate_conditional_drawdown_risk(bt['strategy_returns'], 0.05)
dd = openquant.viz.prepare_drawdown_payload(bt['timestamps'], bt['equity_curve'])

plt.figure(figsize=(10,3.2))
plt.plot(dd['equity'], label='equity')
plt.plot(dd['drawdown'], label='drawdown')
plt.title('Chapter 14: Equity and Drawdown')
plt.legend()
plt.tight_layout()
plt.show()

print({'VaR95': var95, 'ES95': es95, 'CDaR95': cdar95})


{'VaR95': -0.005920041004613053, 'ES95': -0.009514433369964263, 'CDaR95': 0.0329213040792226}


## Interpretation
Risk gating should occur on returns actually implied by signals and execution assumptions, not on model logits in isolation.

